# 💬 Basic Text Classification
## 🚙 Importing libraries, data sets, required files and modules

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import tensorflow_hub as tfhub
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

## ⏬ Data Loading
> 🎈 For datset [details](https://www.tensorflow.org/datasets/catalog/imdb_reviews)

In [0]:
train_val_split_ratio = tfds.Split.TRAIN.subsplit([6, 4])

In [0]:
(train, val), test = tfds.load(
    name = 'imdb_reviews',
    split = (train_val_split_ratio, tfds.Split.TEST),
    as_supervised = True
) 

### 🕵️‍♀️ Understand the data

In [0]:
train_examples, train_labels = next(iter(train.batch(10)))

In [10]:
train_examples

<tf.Tensor: id=459, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [11]:
train_labels

<tf.Tensor: id=460, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

## 👩‍🔧 Data Preparation (pre-processing)

### 🎳 Prepare word embeddings
> To represent words 

In [0]:
embeddings = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
emb_layer = tfhub.KerasLayer(
    embeddings, 
    input_shape=[], 
    dtype = tf.string, 
    trainable=True)

In [17]:
# explore embeddings
emb_layer(train_examples[:2])

<tf.Tensor: id=651, shape=(2, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ]],
      dtype=float32)>

## 👩‍💻 Model Designing
### 🧱 Make layers design
- Number of layers and number of units in each layer
- Activation function of each layer

In [0]:
layers = [
          emb_layer,
          Dense(16, activation='relu'),
          Dense(1, activation='sigmoid')
]

### 🚀 Implement your design

In [0]:
model = Sequential(layers)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 🤓 Training
- Keras .fit() [details](https://keras.io/models/model/#fit)

In [27]:
model.fit(train.shuffle(10000).batch(512),
          epochs=20,
          validation_data=val.batch(512))

Epoch 1/20
30/30 [==============================] - 9s 289ms/step - loss: 0.5552 - acc: 0.7235 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 8s 283ms/step - loss: 0.5321 - acc: 0.7439 - val_loss: 0.5351 - val_acc: 0.7409
Epoch 3/20
30/30 [==============================] - 9s 287ms/step - loss: 0.5063 - acc: 0.7632 - val_loss: 0.5132 - val_acc: 0.7566
Epoch 4/20
30/30 [==============================] - 8s 282ms/step - loss: 0.4794 - acc: 0.7843 - val_loss: 0.4882 - val_acc: 0.7739
Epoch 5/20
30/30 [==============================] - 9s 292ms/step - loss: 0.4475 - acc: 0.8043 - val_loss: 0.4638 - val_acc: 0.7887
Epoch 6/20
30/30 [==============================] - 9s 287ms/step - loss: 0.4173 - acc: 0.8225 - val_loss: 0.4408 - val_acc: 0.8022
Epoch 7/20
30/30 [==============================] - 9s 290ms/step - loss: 0.3880 - acc: 0.8383 - val_loss: 0.4196 - val_acc: 0.8127
Epoch 8/20
30/30 [==============================] - 9s 289ms/step - 

## 👮‍♀️ Evaluate the model
- Keras .evaluate() [details](https://keras.io/models/model/#evaluate)

In [28]:
loss, acc = model.evaluate(test.batch(512))

49/49 [==============================] - 6s 123ms/step - loss: 0.3329 - acc: 0.8567


In [29]:
print('Loss:', loss, 'Accuracy', acc)

Loss: 0.3329200106007712 Accuracy 0.85672
